# SWISSIMAGE loader

1) [Production of extents](#production-of-extents) \
    1.1) [Loading and data visualization](#loading-and-data-visualization) \
    1.2) [Add flight_year to points](#add-flight_year-to-points) \
    1.2) [Add flight_year to merged polygons](#add-flight_year-to-merged-polygons) 
2) [Processing of extents](#processing-of-extents) \
    2.1) [Binarization of masks](#binarization-of-masks) \
    2.2) [Splitting images into 512x512 subset](#splitting-images-into-512x512-subset) \
    2.3) [Remove tiles that don't contain landslides](#remove-tiles-that-dont-contain-landslides) \
    2.4) [Sort into subfolders](#sort-into-subfolders)

## Dependencies

In [2]:
import os
import numpy as np
import geopandas as gpd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from copy import deepcopy

## Production of extents

### Loading and data visualization

In [83]:
src_polygons = r"D:\Terranum_SD\99_Data\Landslide\data\Bern_glissements_spontane_shpfiles\BD glissements spontanés\BE_SL_all_aggreg_PQ.shp"
src_points_intersect = r"D:\Terranum_SD\99_Data\Landslide\data\Bern_glissements_spontane_shpfiles\points_landslide_bern_intersect_polygons.gpkg"
src_km_grid = r"D:\Terranum_SD\99_Data\Landslide\data\Bern_glissements_spontane_shpfiles\swissIMAGE_HIST.gpkg"
src_landslides_merged = r"D:\Terranum_SD\99_Data\Landslide\data\Bern_glissements_spontane_shpfiles\landslides_anriss_auslauf_merged.gpkg"

In [76]:
points_intersect = gpd.read_file(src_points_intersect)
print("Number of years: ", len(points_intersect.year.unique()))
print(np.sort(points_intersect.year.unique()))
points_intersect['flight_year'] = np.zeros(len(points_intersect))
print(points_intersect.columns)

Number of years:  24
[1962 1974 1980 1985 1992 1993 1998 1999 2000 2004 2005 2006 2007 2008
 2009 2010 2012 2014 2015 2016 2017 2018 2021 2022]
Index(['RS_ID', 'year', 'outline', 'source', 'ID_alt', 'Anrissmäc', 'Volumen',
       'ID', 'Jahr', 'Datum', 'SHAPE_Leng', 'SHAPE_Area', 'geometry',
       'flight_year'],
      dtype='object')


In [77]:
# load grid
grid = gpd.read_file(src_km_grid)
print(grid.head())
print(grid.flight_year)


                                         flight_year resolution_of_origin  \
0  1926,1943,1946,1954,1969,1975,1976,1981,1987,1...        50,100,,25,10   
1  1926,1943,1946,1954,1955,1956,1969,1975,1976,1...        50,100,,25,10   
2  1926,1943,1946,1955,1956,1969,1975,1976,1981,1...        50,100,,25,10   
3  1926,1931,1946,1951,1962,1965,1969,1971,1975,1...        50,100,,25,10   
4  1926,1943,1946,1955,1956,1969,1975,1976,1981,1...        50,100,,25,10   

      id     left      top    right   bottom  \
0  45727  2670000  1206000  2671000  1205000   
1  46200  2672000  1201000  2673000  1200000   
2  46201  2672000  1200000  2673000  1199000   
3  45024  2667000  1207000  2668000  1206000   
4  46202  2672000  1199000  2673000  1198000   

                                            geometry  
0  MULTIPOLYGON (((2670601.819 1206000, 2671000 1...  
1  MULTIPOLYGON (((2672000 1201000, 2673000 12010...  
2  MULTIPOLYGON (((2672000 1199083.191, 2672000 1...  
3  MULTIPOLYGON (((2667000 1

### Add flight_year to points

In [ ]:
lst_points_too_recent = []
lst_flight_years = np.zeros(len(points_intersect), dtype=int)
for point in tqdm(points_intersect.itertuples(), total=len(points_intersect)):
    intersection_mask = grid.geometry.contains(point.geometry)#.intersection(grid.geometry)
    intersection = grid[intersection_mask]

    lst_years = []
    if np.sum(intersection_mask) != 1:
        print("Problem (no matching polygon) with : ", point)
        continue

    try:
        flight_year = np.min([int(x) for x in intersection.flight_year.values[0].split(',') if int(x) > point.year])
    except:
        lst_points_too_recent.append(point)
        continue

    lst_flight_years[point.Index] = flight_year

points_intersect['flight_year'] = lst_flight_years

100%|██████████| 556/556 [00:01<00:00, 332.99it/s]


In [ ]:
print("Number of points with year too high for the grid: ", np.sum(lst_flight_years == 0))
print("Different flight years: ", set(points_intersect.flight_year.values))

Number of points with year too high for the grid:  16
Different flight years:  {0, 1968, 1974, 1980, 1985, 1992, 1993, 1998, 1999, 2000, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2015, 2016, 2018, 2021}


In [81]:
points_intersect.to_file(src_points_intersect.split('.gpkg')[0] + '_w_flight_year.gpkg', driver="GPKG")

### Add flight_year to merged polygons

In [ ]:
merged_polygons = gpd.read_file(src_landslides_merged)
# print(merged_polygons.index)points_intersect = gpd.read_file(src_points_intersect.split('.gpkg')[0] + '_w_flight_year.gpkg')
points_intersect = points_intersect[points_intersect.flight_year >= 1998]
flight_year = np.zeros(len(merged_polygons), dtype=int)
list_polygons_too_old = []
for polygon in tqdm(merged_polygons.itertuples(), total=len(merged_polygons)):
    points = [p for p in points_intersect.itertuples() if polygon.geometry.contains(p.geometry)]
    flight_years = [x.flight_year for x in points]
    if len(set(flight_years)) == 1:
        flight_year[polygon.Index] = flight_years[0]
    elif len(set(flight_years)) > 1:
        flight_year[polygon.Index] = np.bincount(flight_years).argmax()
        print([x.flight_year for x in points])
        print(flight_year[polygon.Index])
    else:
        list_polygons_too_old.append(polygon)

  9%|▉         | 44/498 [00:00<00:02, 213.68it/s]

[2010, 2021]
2010
[2004, 1999]
1999


 23%|██▎       | 113/498 [00:00<00:01, 224.44it/s]

[2004, 1998]
1998
[1999, 2007]
1999


 37%|███▋      | 183/498 [00:00<00:01, 225.68it/s]

...

[2007, 2004, 2007]
2007
[1998, 2007]
1998
[2004, 2018]
2004
[2015, 2018]
2015
[2012, 2009]
2009


 83%|████████▎ | 414/498 [00:01<00:00, 220.24it/s]

[2006, 2009, 2015, 2015]
2015
[2006, 2015]
2006
[2015, 2018]
2015
[2004, 2015]
2004
[2012, 2012, 2015]
2012


100%|██████████| 498/498 [00:02<00:00, 220.65it/s]

[2004, 2004, 2000]
2004


In [123]:
print("Number of polygons too old: ", len(list_polygons_too_old))
print("Different years: ", sorted(set(merged_polygons.flight_year)))

Number of polygons too old:  49
Different years:  [0, 1998, 1999, 2000, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2015, 2016, 2018, 2021]


In [120]:
merged_polygons['flight_year'] = flight_year
merged_polygons.to_file(src_landslides_merged.split('.gpkg')[0] + "_w_flight_year.gpkg", driver="GPKG")

## Processing of extents

In [5]:
src_data = r"D:\Terranum_SD\99_Data\Landslide\data\Bern_glissements_spontane_shpfiles\Extents"
src_tiles = r"D:\Terranum_SD\99_Data\Landslide\data\Bern_glissements_spontane_shpfiles\Extents\tiles"
src_no_landslide = os.path.join(src_tiles, "no_landslide")
src_images = os.path.join(src_tiles, 'images')
src_masks = os.path.join(src_tiles, 'labels')
src_masks_bin = os.path.join(src_tiles, 'masks')

### Binarization of masks

In [8]:

list_images_to_process = [os.path.join(src_data, x) for x in os.listdir(src_data) if x.split('.')[-1].lower() in ['png','jpg','jpeg'] and 'mask_bin' not in x]
list_masks = [x for x in list_images_to_process if 'mask' in x]

In [9]:
for _, mask in tqdm(enumerate(list_masks),total=len(list_masks)):
    img  = Image.open(mask)
    img_arr = np.array(img)
    img_arr_rgb = img_arr[..., 0:3]
    img_bin = np.sum(img_arr_rgb, axis=2)
    img_bin[img_bin != 0] = 1
    result = Image.fromarray(img_bin.astype(np.uint8))
    result_src = ''.join(mask.replace("mask", "bin").split('.')[0:-1]) + '.tif'
    result.save(result_src)
    list_images_to_process.append(result_src)
    

100%|██████████| 449/449 [00:26<00:00, 16.87it/s]


### Splitting images into 512x512 subset

In [ ]:
def tile_image_overlap(in_path, tile_size=512, overlap=0, save_dir=None, verbose=False):
    save_dir = os.path.join(os.path.dirname(in_path), 'tiles') if save_dir == None else save_dir
    ext = in_path.split('.')[-1]
    os.makedirs(save_dir, exist_ok=True)

    img = Image.open(in_path)
    w, h = img.size

    tile_id = 0
    for y in range(0, h, tile_size):
        for x in range(0, w, tile_size):
            # manage if borders reached
            if x + tile_size > w:
                x = w - tile_size
            if y + tile_size > h:
                y = h - tile_size

            # Crop region (handles border tiles automatically)
            tile = img.crop((x, y, x + tile_size, y + tile_size))
            output_path = os.path.join(save_dir, ''.join(os.path.basename(in_path).split('.')[:-1]) + f"_{tile_id}.tif")
            tile.save(output_path)
            tile_id += 1
    if verbose:
        print(f"Saved {tile_id} overlapping tiles.")
    return tile_id

In [ ]:
num_samples = 0
# temp_list = [os.path.join(src_data, x) for x in os.listdir(src_data) if x.endswith('tif')]
for _ ,image in tqdm(enumerate(list_images_to_process), total=len(list_images_to_process)):
    num_samples += tile_image_overlap(image)

print("Number of created tiles: ", num_samples)

100%|██████████| 449/449 [00:04<00:00, 110.13it/s]

Number of created tiles:  4685


### Remove tiles that don't contain landslides

In [ ]:
list_masks = [os.path.join(src_tiles, x) for x in os.listdir(src_tiles) if 'mask' in x and not 'bin' in x]
list_masks_bin = [os.path.join(src_tiles, x) for x in os.listdir(src_tiles) if 'mask_bin' in x]

In [28]:

print("Number of masks: ", len(list_masks))
print("Number of masks bin: ", len(list_masks_bin))
# assert len(list_masks) == len(list_masks_bin)

Number of masks:  4685
Number of masks bin:  4685


In [29]:
lst_no_correspondance = []
for mask in list_masks:
    if not os.path.exists(mask.replace('mask', 'image')):
        lst_no_correspondance.append(mask)
if len(lst_no_correspondance) > 0:
    print("The following masks don't have corresponding image:")
    for x in lst_no_correspondance:
        print(f"\t{x}")

In [ ]:
os.makedirs(src_no_landslide, exist_ok=True)

for _, mask_bin in tqdm(enumerate(list_masks_bin), total=len(list_masks_bin)):
    img = Image.open(mask_bin)
    img_arr = np.array(img)
    # img_arr_rgb = img_arr[...,0:3]
    # img_arr_rgb[img_arr_rgb != 0] = 1
    assert set(img_arr.flatten()) in [{0}, {1}, {0, 1}]
    img.close()
    if set(img_arr.flatten()) == {0}:
        # mask = [x for x in list_masks if x.split('.')[0].replace('mask', 'mask_bin') in mask_bin][0]
        mask = mask_bin.replace('_bin', '').replace('tif', 'png')
        image = mask.replace('mask', 'image')
        os.rename(mask_bin, os.path.join(src_no_landslide, os.path.basename(mask_bin)))
        os.rename(mask, os.path.join(src_no_landslide, os.path.basename(mask)))
        os.rename(image, os.path.join(src_no_landslide, os.path.basename(image)))


100%|██████████| 4685/4685 [02:30<00:00, 31.09it/s]


### Sort into subfolders

In [7]:
os.makedirs(src_images, exist_ok=True)
os.makedirs(src_masks, exist_ok=True)
os.makedirs(src_masks_bin, exist_ok=True)

In [22]:
list_tiles = [os.path.join(src_tiles, x) for x in os.listdir(src_tiles) if not os.path.isdir(os.path.join(src_tiles,x))]

print(len(list_tiles))

for _, tile_src in tqdm(enumerate(list_tiles), total=len(list_tiles)):
    if 'image' in tile_src:
        os.rename(tile_src, os.path.join(src_images, os.path.basename(tile_src).replace('image_', '')))
    elif 'mask' in tile_src:
        os.rename(tile_src, os.path.join(src_masks, os.path.basename(tile_src).replace('mask_', '')))
    elif 'bin' in tile_src:
        os.rename(tile_src, os.path.join(src_masks_bin, os.path.basename(tile_src).replace('bin_', '')))
    else:
        print(f"No category for: {tile_src}")

0


0it [00:00, ?it/s]


# Trash

In [ ]:
list_masks_bin = [os.path.join(src_no_landslide, x) for x in os.listdir(src_no_landslide) if "mask_bin" in x and x.endswith('.tif')]
print(len(list_masks_bin))
for mask in list_masks_bin:
    os.rename(mask, mask.replace('mask_bin', 'bin'))

1124


In [ ]:
list_img = [os.path.join(src_no_landslide, x) for x in os.listdir(src_no_landslide) if not os.path.isdir(os.path.join(src_no_landslide,x)) and ".tif" not in x]
print(len(list_img))
for _, img_src in tqdm(enumerate(list_img), total=len(list_img)):
    img = Image.open(img_src)
    img.save(''.join(img_src.split('.')[:-1]) + '.tif')
    os.remove(img_src)

2248


100%|██████████| 2248/2248 [00:27<00:00, 82.36it/s] 
